<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/CRISPR_AAI_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y google-generativeai
!pip install google-generativeai

Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 2.9 MB/s eta 0:00:00


In [7]:
import google.generativeai as genai
from google.colab import userdata
import json
import time
import random

# --- 0. Gemini API Setup ---
try:
    # >>> IMPORTANT PRE-REQUISITE STEPS:
    # >>> 1. Disconnect and delete your current Colab runtime (Runtime -> Disconnect and delete runtime).
    # >>> 2. In a BRAND NEW Colab session, in the VERY FIRST cell, RUN THE FOLLOWING COMMANDS:
    # >>>    !pip uninstall -y google-generativeai
    # >>>    !pip install google-generativeai
    # >>> 3. THEN, IMMEDIATELY RESTART YOUR COLAB RUNTIME (Runtime -> Restart runtime...).
    # >>> 4. AFTER RESTART, RUN ALL CELLS FROM THE BEGINNING.
    # >>> This is crucial for 'genai.types.ToolOutput' to be available and correctly loaded.

    GOOGLE_API_KEY = userdata.get('GEMINI')
    genai.configure(api_key=GOOGLE_API_KEY)
    gemini_model = genai.GenerativeModel('gemini-1.5-pro-latest')
    print("Gemini 1.5 Pro model initialized successfully.")
except Exception as e:
    print(f"Error initializing Gemini model: {e}")
    print("Please ensure your 'GEMINI' API key is set in Colab secrets and you've followed the library update/restart steps carefully.")
    exit()


# --- 1. Simulated External "Tools" for CRISPR Operations ---

def search_scientific_literature(query: str, num_results: int = 3) -> str:
    """
    Simulates searching scientific databases for CRISPR-related information.
    Args:
        query (str): The search query (e.g., "Cas9 efficiency", "CRISPR off-target effects").
        num_results (int): Number of simulated results to return.
    Returns:
        str: A summary of simulated search results.
    """
    print(f"\n🔬 [Tool Call] Searching literature for: '{query}'...")
    time.sleep(1.0) # Simulate network latency
    results = []
    if "HTT gene knockout hiPSC" in query.lower():
        results.append("Paper 1: 'High-efficiency HTT gene knockout in hiPSCs using optimized gRNAs and electroporation.'")
        results.append("Paper 2: 'Minimizing off-target effects in hiPSC gene editing with high-fidelity Cas9 variants.'")
        results.append("Paper 3: 'Comparative analysis of viral vs. non-viral delivery for hiPSC CRISPR.'")
    elif "Cas9 efficiency" in query.lower():
        results.append("Review: 'Advancements in Cas9 engineering for enhanced specificity and activity.'")
        results.append("Study: 'PAM sequence variants influence Cas9 cleavage efficiency.'")
    elif "CRISPR off-target" in query.lower():
        results.append("Tool Paper: 'CRISPR-TOP: A novel algorithm for predicting and visualizing off-target sites.'")
        results.append("Review: 'Strategies for mitigating off-target effects in therapeutic gene editing.'") # Corrected in last step
    else:
        results.append(f"Generic literature result for '{query}'.")

    return "\n".join(results[:int(num_results)]) # Ensure num_results is an int for slicing

def design_guide_rna(target_gene: str, species: str, num_candidates: int = 3) -> str:
    """
    Simulates a sophisticated bioinformatics tool for designing optimal guide RNAs.
    Args:
        target_gene (str): The gene name to target (e.g., "HTT", "CFTR").
        species (str): The target species (e.g., "human", "mouse").
        num_candidates (int): Number of gRNA candidates to generate.
    Returns:
        str: A JSON string of candidate gRNAs with scores.
    """
    print(f"\n🧬 [Tool Call] Designing gRNAs for gene: {target_gene} in {species}...")
    time.sleep(2.0)
    gRNA_candidates = []
    for i in range(num_candidates):
        seq_prefix = "".join(random.choices("ACGT", k=10))
        gRNA_seq = f"{seq_prefix}{target_gene.upper()}_gRNA{i+1}"
        specificity = round(random.uniform(0.85, 0.99), 2)
        efficiency = round(random.uniform(0.70, 0.95), 2)
        off_targets = []
        if specificity < 0.95 and random.random() < 0.5: # Simulate some off-targets for lower specificity
            off_targets = [f"Chr{random.randint(1,22)}:{random.randint(10000,99999)}",
                           f"Chr{random.randint(1,22)}:{random.randint(10000,99999)}"]

        gRNA_candidates.append({
            "sequence": gRNA_seq,
            "specificity_score": specificity,
            "on_target_efficiency_score": efficiency,
            "predicted_off_target_sites": off_targets
        })
    return json.dumps(gRNA_candidates)

def simulate_gene_editing_outcome(gRNA_json: str, cas_variant: str, cell_type: str, delivery_method: str) -> str:
    """
    Simulates the outcome of a gene-editing experiment based on design parameters.
    Args:
        gRNA_json (str): JSON string of the selected gRNA.
        cas_variant (str): The Cas enzyme variant (e.g., "Cas9", "Cas9-HF1").
        cell_type (str): The cell type (e.g., "hiPSC", "HEK293").
        delivery_method (str): The method of delivering CRISPR components (e.g., "electroporation", "lentivirus").
    Returns:
        str: A summary of the simulated outcome.
    """
    print(f"\n🧪 [Tool Call] Simulating gene editing for {cell_type} with {cas_variant} via {delivery_method}...")
    time.sleep(2.5)
    try:
        gRNA_details = json.loads(gRNA_json)
    except json.JSONDecodeError:
        return "Error: Invalid gRNA JSON provided to simulation tool."

    base_efficiency = gRNA_details.get('on_target_efficiency_score', 0)
    specificity = gRNA_details.get('specificity_score', 0)
    predicted_off_target_sites = gRNA_details.get('predicted_off_target_sites', [])


    # Adjust efficiency/specificity based on Cas variant and delivery method (simplified logic)
    if "HF1" in cas_variant or "eCas9" in cas_variant:
        specificity = min(1.0, specificity * 1.05) # Slightly boost specificity
    if "lentivirus" in delivery_method.lower() and "hiPSC" in cell_type.lower():
        base_efficiency = min(1.0, base_efficiency * 1.1) # Lentivirus often good for iPSCs
    elif "electroporation" in delivery_method.lower() and "hiPSC" in cell_type.lower():
        base_efficiency = base_efficiency * 0.95 # Can be a bit harsher on viability

    final_ko_efficiency = round(base_efficiency * random.uniform(0.9, 1.0), 2)
    final_off_target_count = len(predicted_off_target_sites)

    # Introduce some randomness for simulation realism
    if random.random() < 0.15 and final_off_target_count == 0: # Small chance of unpredicted off-target
        final_off_target_count = random.randint(1, 2)
        # Note: We don't modify the original gRNA_details here, just report the simulated outcome
        # predicted_off_target_sites.append("NEW_UNPREDICTED_SITE")


    if final_off_target_count > 0:
        outcome_summary = (f"Simulation complete: Predicted knockout efficiency: {final_ko_efficiency*100}%. "
                           f"WARNING: {final_off_target_count} potential off-target sites detected. "
                           "Further optimization or validation recommended.")
    else:
        outcome_summary = (f"Simulation complete: Predicted knockout efficiency: {final_ko_efficiency*100}%. "
                           "No significant off-target effects predicted.")

    return outcome_summary

def generate_experimental_protocol(details_json: str) -> str:
    """
    Simulates generating a detailed, step-by-step laboratory protocol.
    Args:
        details_json (str): JSON string containing all necessary protocol details.
    Returns:
        str: A formatted experimental protocol.
    """
    print(f"\n📝 [Tool Call] Generating experimental protocol...")
    time.sleep(1.5)
    try:
        details = json.loads(details_json)
    except json.JSONDecodeError:
         return "Error: Invalid protocol details JSON provided to protocol generation tool."


    protocol = f"""
--- EXPERIMENTAL PROTOCOL ---
Objective: {details.get('objective', 'CRISPR Gene Editing')}
Target Gene: {details.get('target_gene', 'N/A')}
Guide RNA (gRNA): {details.get('gRNA_sequence', 'N/A')} (Specificity: {details.get('gRNA_specificity', 'N/A')}, Efficiency: {details.get('gRNA_efficiency', 'N/A')})
Cas Enzyme: {details.get('cas_variant', 'Cas9')}
Cell Type: {details.get('cell_type', 'N/A')}
Delivery Method: {details.get('delivery_method', 'N/A')}

**I. Reagents & Materials:**
- {details.get('cas_variant', 'Cas9')} nuclease
- {details.get('gRNA_sequence', 'N/A')} (synthetic or plasmid)
- {details.get('cell_type', 'N/A')} culture
- Appropriate cell culture media, supplements ({details.get('cell_media', 'e.g., DMEM/F12, mTeSR1')})
- {details.get('delivery_method', 'N/A')} kit/reagents ({details.get('delivery_kit_notes', 'e.g., Lonza Nucleofector Kit, Lipofectamine')})
- Sterile consumables (plates, tips, tubes)
- Genomic DNA extraction kit
- PCR reagents, primers for validation
- NGS library preparation reagents (if applicable)

**II. Procedure:**
1.  **Cell Preparation:**
    * Culture {details.get('cell_type', 'N/A')} cells to {details.get('cell_confluence', '70-80%')} confluence.
    * Harvest cells and count accurately. Aim for {details.get('cell_count_per_transfection', '5x10^5 - 1x10^6')} cells per transfection.
2.  **RNP Complex/Plasmid Preparation:**
    * For RNP: Mix {details.get('cas_variant', 'Cas9')} and gRNA at a molar ratio of {details.get('rnp_ratio', '1:1.2')} in appropriate buffer. Incubate at 37°C for 10-15 min.
    * For Plasmid: Prepare plasmid DNA solution according to {details.get('delivery_method', 'N/A')} manufacturer's instructions.
3.  **Transfection/Delivery:**
    * Follow the manufacturer's protocol for {details.get('delivery_method', 'N/A')} specific to {details.get('cell_type', 'N/A')}.
    * Typical parameters for {details.get('delivery_method', 'N/A')}: [Specific voltage/time for electroporation, or reagent-to-DNA ratio for lipofection].
4.  **Post-Transfection Recovery:**
    * Immediately transfer cells to pre-warmed complete growth media containing {details.get('recovery_supplements', 'e.g., ROCK inhibitor')} for {details.get('recovery_duration', '24 hours')}.
    * Change media to regular growth media after {details.get('recovery_duration', '24 hours')}.
5.  **Validation ({details.get('validation_methods', 'PCR, Western Blot, NGS')}):**
    * Harvest genomic DNA and/or protein {details.get('validation_time_point', '48-72 hours')} post-transfection.
    * Perform {details.get('validation_methods', 'PCR-based assays (e.g., T7E1), Sanger sequencing, or Next-Generation Sequencing (NGS)')} to confirm gene editing events.
    * Perform {details.get('protein_validation_methods', 'Western Blot')} to confirm protein knockout.
    * For off-target analysis, conduct {details.get('off_target_analysis_methods', 'whole-genome sequencing or targeted NGS of predicted off-target sites')}.
    * For hiPSCs, {details.get('additional_hiPSC_validation', 'Karyotyping is essential to confirm genomic integrity after manipulation.')}

**III. Data Analysis & Interpretation:**
- Analyze sequencing data to determine precise edits and efficiency.
- Compare protein levels to unedited controls.
- Review off-target sequencing data carefully.

**IV. Safety & Disposal:**
- Always use sterile technique.
- Dispose of all biological and chemical waste according to institutional guidelines.
--- END PROTOCOL ---
    """
    return protocol

# --- 2. Define Tools for Gemini 1.5 Pro Function Calling ---
tools = [
    {
        "function_declarations": [
            {
                "name": "search_scientific_literature",
                "description": "Searches a simulated scientific literature database for relevant papers and reviews.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {"type": "string", "description": "The scientific query to search for."},
                        "num_results": {"type": "integer", "description": "Number of top results to return."}
                    },
                    "required": ["query"]
                }
            }
        ]
    },
    {
        "function_declarations": [
            {
                "name": "design_guide_rna",
                "description": "Designs candidate guide RNA (gRNA) sequences for a target gene in a specified species, providing predicted scores.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "target_gene": {"type": "string", "description": "The name of the gene to target (e.g., 'HTT')."},
                        "species": {"type": "string", "description": "The species of the target genome (e.g., 'human', 'mouse')."},
                        "num_candidates": {"type": "integer", "description": "Number of gRNA candidates to generate."}
                    },
                    "required": ["target_gene", "species"]
                }
            }
        ]
    },
    {
        "function_declarations": [
            {
                "name": "simulate_gene_editing_outcome",
                "description": "Simulates the outcome of a gene-editing experiment based on design parameters.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "gRNA_json": {"type": "string", "description": "JSON string of the selected gRNA details (from design_guide_rna)."},
                        "cas_variant": {"type": "string", "description": "The Cas enzyme variant to use (e.e.g., 'Cas9', 'Cas9-HF1')."},
                        "cell_type": {"type": "string", "description": "The cell type for the experiment (e.g., 'hiPSC', 'HEK293')."},
                        "delivery_method": {"type": "string", "description": "The method of delivering CRISPR components (e.g., 'electroporation', 'lentivirus')."}
                    },
                    "required": ["gRNA_json", "cas_variant", "cell_type", "delivery_method"]
                }
            }
        ]
    },
    {
        "function_declarations": [
            {
                "name": "generate_experimental_protocol",
                "description": "Generates a detailed, step-by-step laboratory protocol for a CRISPR experiment.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "details_json": {"type": "string", "description": "JSON string containing all parameters for the protocol (target_gene, gRNA_sequence, cas_variant, cell_type, delivery_method, validation_methods, etc.)."}
                    },
                    "required": ["details_json"]
                }
            }
        ]
    }
]

# Map tool names to their actual Python functions
available_tools = {
    "search_scientific_literature": search_scientific_literature,
    "design_guide_rna": design_guide_rna,
    "simulate_gene_editing_outcome": simulate_gene_editing_outcome,
    "generate_experimental_protocol": generate_experimental_protocol
}

# --- 3. The Agentic AI Core ---

class CrisprOptimizeAgent:
    def __init__(self, llm_model: genai.GenerativeModel, tools_list: list, available_tools_map: dict):
        self.llm = llm_model
        self.tools = tools_list
        self.available_tools_map = available_tools_map
        self.chat_history = []
        self.state = {} # Agent's internal memory/state for the current task

    def _add_to_history(self, role: str, parts: list):
        self.chat_history.append({"role": role, "parts": parts})

    def _call_tool(self, tool_name: str, **kwargs):
        """Executes a tool and returns its output."""
        print(f"\n📞 [Agent] Calling tool: {tool_name} with args: {kwargs}")

        # Explicitly cast arguments based on tool definitions to handle model generating wrong types
        processed_kwargs = {}
        tool_declarations = next((t['function_declarations'] for t in self.tools if t['function_declarations'][0]['name'] == tool_name), None)

        if tool_declarations and tool_declarations[0]['parameters'] and 'properties' in tool_declarations[0]['parameters']:
            param_properties = tool_declarations[0]['parameters']['properties']
            for arg_name, arg_value in kwargs.items():
                if arg_name in param_properties:
                    expected_type = param_properties[arg_name].get('type')
                    if expected_type == 'integer' and isinstance(arg_value, float):
                        print(f"ℹ️ [Tool Call Processing] Casting float '{arg_name}': {arg_value} to integer.")
                        processed_kwargs[arg_name] = int(arg_value)
                    elif expected_type == 'number' and isinstance(arg_value, (int, float)):
                         processed_kwargs[arg_name] = float(arg_value)
                    # Add other type conversions if necessary (e.g., string, boolean)
                    else:
                        processed_kwargs[arg_name] = arg_value
                else:
                    processed_kwargs[arg_name] = arg_value
        else:
            processed_kwargs = kwargs # No specific type info, use as is


        if tool_name in self.available_tools_map:
            func = self.available_tools_map[tool_name]
            try:
                result = func(**processed_kwargs) # Use processed arguments
                print(f"✅ [Tool Call Success] '{tool_name}' returned: {result[:150]}...")
                return result
            except Exception as e:
                print(f"❌ [Tool Call Error] Exception during '{tool_name}' execution: {e}")
                # Return an error message as tool output so the model can potentially handle it
                return f"Error executing tool '{tool_name}': {e}"
        else:
            print(f"❌ [Tool Call Error] Tool '{tool_name}' not found.")
            return f"Error: Tool '{tool_name}' not found."


    def run_task(self, user_goal: str):
        print(f"\n🚀 AI Agent Initiated for Goal: {user_goal} 🚀")
        self.state = {"user_goal": user_goal}
        self._add_to_history("user", [{"text": user_goal}])

        while True:
            try:
                print("\n🧠 [Agent] Sending history to LLM for next step...")
                response = self.llm.generate_content(
                    self.chat_history,
                    tools=self.tools,
                    tool_config={"function_calling_config": {"mode": "AUTO"}}
                )
                print("✅ [Agent] Received response from LLM.")


                if not response.candidates:
                    print("\n⚠️ [Agent] No valid response candidates from LLM (e.g., blocked by safety settings or no relevant output). Ending task.")
                    if getattr(response, 'prompt_feedback', None) and response.prompt_feedback.block_reason:
                         print(f"🚫 [Agent] Block reason: {response.prompt_feedback.block_reason}")
                    break

                candidate = response.candidates[0]

                function_calls_in_content = [
                    part.function_call for part in candidate.content.parts
                    if getattr(part, 'function_call', None)
                ]

                if function_calls_in_content:
                    function_call = function_calls_in_content[0]
                    tool_name = function_call.name
                    tool_args = {k: v for k, v in function_call.args.items()}

                    print(f"\n🤖 [Agent Decision] LLM wants to call tool: {tool_name}") # Args printed in _call_tool

                    # Add the function call to history before executing the tool
                    self._add_to_history("model", [function_call])

                    tool_output = self._call_tool(tool_name, **tool_args)

                    # --- Workaround: NOT adding tool output to history to avoid "invalid role" error ---
                    # The previous error "Please use a valid role: user, model." indicates
                    # the API/model might not support the 'tool' role in history.
                    # Not adding tool output to history prevents this specific error,
                    # but severely limits the agent's ability to reason with tool results.
                    print("\n⚠️ [Agent Workaround] Tool output NOT added to chat history due to persistent 'invalid role' error.")
                    # --- End Workaround ---

                    # --- Process tool output and update state (relies on tool output string) ---
                    try:
                        if tool_name == "design_guide_rna":
                            print("\n✨ [Agent Processing] Processing design_guide_rna output...")
                            # Attempt to parse if the output looks like gRNA json
                            if isinstance(tool_output, str) and tool_output.strip().startswith('['):
                                self.state["gRNA_candidates_json"] = tool_output # Store raw JSON
                                try:
                                    gRNA_candidates = json.loads(tool_output)
                                    if gRNA_candidates:
                                        # Simple selection logic based on efficiency
                                        best_gRNA = max(gRNA_candidates, key=lambda x: x.get('on_target_efficiency_score', 0))
                                        self.state["selected_gRNA_details"] = best_gRNA # Store parsed details
                                        print(f"\n🌟 [Agent State Update] Selected gRNA: {best_gRNA.get('sequence', 'N/A')} (Specificity: {best_gRNA.get('specificity_score', 'N/A')}, Efficiency: {best_gRNA.get('on_target_efficiency_score', 'N/A')})")
                                    else:
                                        print("\n⚠️ [Agent State Update] No gRNA candidates were designed.")
                                        self.state["selected_gRNA_details"] = None
                                except json.JSONDecodeError as e:
                                     print(f"\n❌ [Agent Processing Error] Could not parse gRNA candidates output as JSON: {e}")
                                     self.state["gRNA_candidates_json"] = None
                                     self.state["selected_gRNA_details"] = None
                            else:
                                print("\n⚠️ [Agent Processing] design_guide_rna output not in expected JSON format.")
                                self.state["gRNA_candidates_json"] = None
                                self.state["selected_gRNA_details"] = None


                        elif tool_name == "simulate_gene_editing_outcome":
                            print("\n✨ [Agent Processing] Processing simulate_gene_editing_outcome output...")
                            self.state["simulation_result"] = tool_output
                            print(f"\n📊 [Agent State Update] Simulation Result: {tool_output}")

                        elif tool_name == "generate_experimental_protocol":
                            print("\n✨ [Agent Processing] Processing generate_experimental_protocol output...")
                            self.state["final_protocol"] = tool_output
                            print(f"\n📜 [Agent State Update] Final Protocol Generated.")
                            print("\n" + tool_output)
                            print("\n--- Task Completed by AI Agent ---")
                            break # Task finished

                    except Exception as e:
                        print(f"\n❌ [Agent Processing Error] An error occurred during state update after tool call: {e}")
                        # Decide if this error should stop the agent or just log a warning.
                        # For now, let's log and continue, hoping the model can recover or the next step works.
                        pass # Continue the loop despite state update error


                elif candidate.text:
                    print(f"\n💬 [Agent Response] {candidate.text}")
                    self._add_to_history("model", [{"text": candidate.text}])

                    if "task complete" in candidate.text.lower() or "final report" in candidate.text.lower() or "finished" in candidate.text.lower():
                        print("\n--- Task Completed by AI Agent ---")
                        break

                    print("\n❓ [Agent] LLM provided a text response but no tool call. Assuming current sub-task completion or awaiting further instruction.")
                    # If the LLM provides text and no tool call, it might indicate it's done or stuck.
                    # For this agent, let's assume it's done if it gives a text response without a tool call after the initial step.
                    # In a more complex agent, you might have logic here to decide if the task is truly complete.
                    break


            except genai.types.BlockedPromptException as e:
                print(f"\n🚫 [Agent Error] Prompt was blocked by safety settings: {e}")
                if getattr(e, 'response', None) and getattr(e.response, 'prompt_feedback', None):
                     print(f"🚫 [Agent] Block reason: {e.response.prompt_feedback.block_reason}")
                break
            except Exception as e:
                print(f"\n❌ [Agent Error] An unexpected error occurred during agent execution: {e}")
                break

# --- Running the CRISPR Optimization Agent ---
if __name__ == "__main__":
    crispr_agent = CrisprOptimizeAgent(gemini_model, tools, available_tools)

    user_goal_crispr = (
        "Optimize HTT gene knockout in human induced pluripotent stem cells (hiPSCs) "
        "for Huntington's disease modeling. "
        "The optimization should focus on minimizing off-target effects and maximizing knockout efficiency. "
        "Once optimized, generate a detailed laboratory protocol."
    )

    crispr_agent.run_task(user_goal_crispr)

    print("\n--- End of CRISPR Agent Demonstration ---")

Gemini 1.5 Pro model initialized successfully.

🚀 AI Agent Initiated for Goal: Optimize HTT gene knockout in human induced pluripotent stem cells (hiPSCs) for Huntington's disease modeling. The optimization should focus on minimizing off-target effects and maximizing knockout efficiency. Once optimized, generate a detailed laboratory protocol. 🚀

🧠 [Agent] Sending history to LLM for next step...
✅ [Agent] Received response from LLM.

🤖 [Agent Decision] LLM wants to call tool: search_scientific_literature

📞 [Agent] Calling tool: search_scientific_literature with args: {'query': "HTT gene knockout hiPSC Huntington's disease minimize off-target effects maximize knockout efficiency", 'num_results': 5.0}
ℹ️ [Tool Call Processing] Casting float 'num_results': 5.0 to integer.

🔬 [Tool Call] Searching literature for: 'HTT gene knockout hiPSC Huntington's disease minimize off-target effects maximize knockout efficiency'...
✅ [Tool Call Success] 'search_scientific_literature' returned: Generic 